In [29]:
import xml.etree.ElementTree as ET
import re
import pandas as pd 
from bs4 import BeautifulSoup as bs

In [30]:
# open file in read mode 
file = open("ipa200409.xml", "r") 
file_content_raw = file.read()
file.close()
text1 = re.compile("<\?xml version\=\"1\.0\" encoding\=\"UTF\-8\"\?>")
file_content = text1.split(file_content_raw)

while "" in file_content:
  file_content.remove('')
  print("No. of patents : ", len(file_content))

No. of patents :  5804


In [31]:
dic = {
    'juridiction' : [],
    'pub_year' : [],
    'kind' : [],
    'app_year' : [],
    'app_type' : [],
    'prior_year' : [], 
    'prior_country' : [],
    'title' : [],
    'assignee' : [],
    'inventors' : [],
    'abstract' : [],
    'claims' : []
}


In [32]:

for i, file in enumerate(file_content):
#i = file_content[10]
    root = ET.fromstring(str(bs(file, 'xml')))
    if root.tag == 'us-patent-application':
        # getting country of publication from attributes of the roots 
        dic['juridiction'].append(root.attrib['country'])

        # giving name to childs of root 
        bibliographic_data_elem = root.find('us-bibliographic-data-application')
        abstract = root.find('abstract')
        drawings = root.find('drawings')
        description = root.find('description')
        claims = root.find("claims")

        # appending abstract to the dic
        dic['abstract'].append(abstract.find('p').text)

        # appending publication year and kind
        for i in bibliographic_data_elem.find('publication-reference').find('document-id').getchildren():
            if i.tag == 'date':
                yyyy = i.text[0:4]
                # mm = i.text[4:6]
                # dd = i.text[6:8]
                dic['pub_year'].append(int(yyyy))
            if i.tag == 'kind':
                dic['kind'].append(i.text)

        # appending application year and application utility
        for i in bibliographic_data_elem.find('application-reference').find('document-id').getchildren():
            if i.tag == 'date':
                yyyy = i.text[0:4]
                # mm = i.text[4:6]
                # dd = i.text[6:8]
                dic['app_year'].append(int(yyyy))

        dic['app_type'].append(bibliographic_data_elem.find('application-reference').attrib['appl-type'])

        # for priority claims/ if published in other countries 
        priority_claims = bibliographic_data_elem.find('priority-claims')
        if priority_claims != None:
            for i in priority_claims.find('priority-claim').getchildren():
                if i.tag == 'country':
                    dic['prior_country'].append(i.text)
                if i.tag == 'date':
                    yyyy = i.text[0:4]
                    dic['prior_year'].append(int(yyyy))
                    #mm = i.text[4:6]
                    #dd = i.text[6:8]
        else:
            dic['prior_country'].append('')
            dic['prior_year'].append('')

        # adding invention title
        dic['title'].append(bibliographic_data_elem.find('invention-title').text)

        # adding assignees
        assignees = bibliographic_data_elem.find('assignees')
        if assignees != None:
            org_names = []
            for assignee in assignees:
                if assignee.find('orgname') != None:
                    org_name = assignee.find('orgname').text
                    org_names.append(org_name)
            dic['assignee'].append(org_names)
        else:
            dic['assignee'].append('')


        # adding Inventors
        us_parties = bibliographic_data_elem.find('us-parties')
        inventors = us_parties.find('inventors').getchildren()
        inventor = []
        #countries = []
        addressbooks = []
        for i in inventors:
            addressbooks.append(i.getchildren()[0])
        for i in addressbooks:
            name = ''
            childs = i.getchildren()
            for child in childs:
                if child.tag == 'last-name':
                    name += child.text
                if child.tag == 'first-name':
                    name = child.text + ' ' + name
                #if child.tag == 'address':
                #    countries.append(child.find('country').text)
            inventor.append(name)
        dic['inventors'].append(inventor)

        # adding claims
        claims_child = claims.getchildren()
        claims = ''
        for childs in claims_child:
            for child in childs:
                for i in child:
                    if i.tag == 'claim-text':
                        if i.text != None:
                            claims += i.text + ' '
        dic['claims'].append(claims)

In [33]:

print(len(dic['juridiction']), len(dic['pub_year']), len(dic['kind']), len(dic['app_year']), len(dic['app_type']),
len(dic['title']), len(dic['assignee']), len(dic['inventors']), len(dic['abstract']), len(dic['prior_country']), len(dic['prior_year']), len(dic['claims']))

5626 5626 5626 5626 5626 5626 5626 5626 5626 5626 5626 5626


In [1]:
import patent_parser as pp
df = pp.parse_patents('ipa200409.xml', 'ipa200409')
df

,juridiction,pub_year,kind,app_year,app_type,prior_year,prior_country,title,assignee,inventors,abstract,claims
0,US,2020,A1,2019,utility,2017,RU,METHOD FOR CULTIVATING SOWN CROPS AND APPARATU...,,[Sergei Vladimirovich BRINDYUK],A method for increasing the efficiency of farm...,"in a technological operation, preparing the fi..."
1,US,2020,A1,2018,utility,2017,FR,HITCH DEVICE FOR AGRICULTURAL VEHICLE,,[Hubert DEFRANCQ],A hitch device for an implement is capable of ...,a first chassis ( a second chassis ( means ( a...
2,US,2020,A1,2019,utility,,,COMPACT ROW CLOSER AND ROW CLEANER ASSEMBLIES,,"[Ronald Steve Martin, Travis Harnetiaux, Asgha...",An apparatus and system for adjustably control...,a frame mounting plate; a set of at least two ...
3,US,2020,A1,2018,utility,,,SYSTEM AND RELATED METHODS FOR MONITORING AND ...,[CNH Industrial America LLC],[Ivan Antich],"In one aspect, a method is disclosed for adjus...","determining, by the computing device, a target..."
4,US,2020,A1,2018,utility,,,SYSTEM AND METHOD FOR CONTROLLING THE SPEED OF...,[CNH Industrial America LLC],"[Christopher Schoeny, Trevor Stanhope, Darian ...","In one aspect, a system for controlling the sp...",a furrow closing assembly including at least o...
...,...,...,...,...,...,...,...,...,...,...,...,...
5621,US,2020,P1,2019,plant,,,Calla lily plant named 'Peter's Pride',,[Peter Beckman],‘Peter's Pride’ is a new variety of calla lily...,
5622,US,2020,P1,2019,plant,,,Calla lily plant named 'Diva Himalaya',,[Peter Beckman],‘Diva Himalaya’ is a new variety of calla lily...,
5623,US,2020,P1,2019,plant,,,Calla Lily plant named 'Diva Alpina',,[Peter Beckman],‘Diva Alpina’ is a new variety of calla lily p...,
5624,US,2020,P1,2019,plant,,,Calla lily plant named 'Diva Siberia',,[Peter Beckman],‘Diva Siberia’ is a new variety of calla lily ...,


In [2]:
df.to_csv('ipa200409.csv', index = False)